In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# read and print the stock tickers that make up S&P500
tickers = pd.read_html(
    'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]

In [ ]:
# starting and ending point of the historical serie
start_date = "2006-01-01"
end_date = "2010-01-01"

# create an array of interested tickers
ticker_symbols = np.array(tickers[tickers['Date added'] < start_date]['Symbol'])
print(f'number of tickers up to now = {len(ticker_symbols)}')

In [ ]:
%%time
#downloading data of the first ticker from yahoo finance
df = yf.download(ticker_symbols[0], start=start_date, end=end_date).drop(
    columns=[ 'Open', 'High', 'Low']).rename(
    
    columns={
        
        'Close': 'close_'+ticker_symbols[0],
        'Adj Close': 'adj_close_'+ticker_symbols[0],
        "Volume": "volume_"+ticker_symbols[0],
        
    })

for i in range(1, len(ticker_symbols)):

    #downloading data from yahoo finance
    dh = yf.download(ticker_symbols[i], start=start_date, end=end_date).drop(
        columns=[ 'Open', 'High', 'Low']).rename(

        columns={

            'Close': 'close_'+ticker_symbols[i],
            'Adj Close': 'adj_close_'+ticker_symbols[i],
            "Volume": "volume_"+ticker_symbols[i],

        })
    
    df = pd.concat([df, dh], axis=1)

In [ ]:
# check about the presence of NaN values in the dataset
print(df.isnull().values.any())

In [ ]:
# check those tickers that have some NaN values and delete them from the dataset
nan_tickers= {}
trading_days = df.shape[0]
for l in ticker_symbols:
    
    if (df['close_'+l].isnull().sum() + df['volume_'+l].isnull().sum() + df['adj_close_'+l].isnull().sum()) > 0:
        
        # percentage of NaN values with respect the total trading days in close, adj_close and volume
        nan_tickers[l] = [
            
            round(100*df['close_'+l].isnull().sum()/trading_days),
            round(100*df['adj_close_'+l].isnull().sum()/trading_days),
            round(100*df['volume_'+l].isnull().sum()/trading_days)
                         ]
delete_tickers = list(nan_tickers.keys())
print(f'number of tickers with NaN values = {len(delete_tickers)}')

In [ ]:
# delete those tickers that have some NaN values
# first build the list of columns to delete
drop_columns = []
for l in delete_tickers:
    drop_columns.append('close_' + l)
    drop_columns.append('adj_close_' + l)
    drop_columns.append('volume_' + l)

df = df.drop(
    columns= drop_columns
)


print(df.isnull().values.any())

In [ ]:
print(f'final number of tickers = {round(df.shape[1]/3)}')
print(f'number of tickers with NaN values = {len(delete_tickers)}')

In [ ]:
# def. of new and final list of tickers
final_tickers = list(ticker_symbols)
for l in delete_tickers:
    final_tickers.remove(l)

In [ ]:
# EXPORT the DATASET as txt file
# OBS. : actually this format is not so useful !

#df.to_csv('data/complete_dataset.txt', index=False, float_format='%.15g')


### Export all the useful datasets

**Obs.**: ChatGPT suggests me to export the datset as txt file instead of csv file. <br>
It says that txt usually preserves better the precision of the numerical values.

In [ ]:
# EXPORT the final list of TICKERS
pd.DataFrame(final_tickers, columns=['ticker']).to_csv('data/ticker.txt', index=False)

In [ ]:
# convert the index to a DatetimeIndex
df.index = pd.to_datetime(df.index)

# convert the date index to a list of strings
date_strings = df.index.date.astype(str).tolist()

# EXPORT the DATES as txt file
pd.DataFrame(date_strings, columns=['date']).to_csv('data/date.txt', index=False)


In [ ]:
# check if the two dataframe (the one exported and the one imported) are the same

print(pd.read_csv('data/date.txt').equals(pd.DataFrame(date_strings, columns=['date'])))

In [ ]:
# EXPORT CLOSE PRICES
df_close = df.iloc[:, ::3] 
df_close.columns = final_tickers
df_close.to_csv('data/close.txt', index=False, float_format='%.15g')

# EXPORT ADJUSTED CLOSE PRICE
df_adj_close = df.iloc[:, 1::3]
df_adj_close.columns = final_tickers
df_adj_close.to_csv('data/adj_close.txt', index=False, float_format='%.15g')

# EXPORT VOLUME
df_volume = df.iloc[:, 2::3]
df_volume.columns = final_tickers
df_volume.to_csv('data/volume.txt', index=False, float_format='%.15g')

In [ ]:
# EXPORT CLOSE PRICES' RETURNS
df_close_return = (df_close / df_close.shift(1) - 1).dropna()
df_close_return.to_csv('data/close_return.txt', index=False, float_format='%.15g')

# EXPORT ADJUSTED CLOSE PRICES' RETURNS
df_adj_close_return = (df_adj_close / df_adj_close.shift(1) - 1).dropna()
df_adj_close_return.to_csv('data/adj_close_return.txt', index=False, float_format='%.15g')

**comparison** between the imported dataset<br>

In [ ]:
# these values of atol and rtol are the standard ones
pd.testing.assert_frame_equal(
    # change the index to the standar one, otherwise it'll raise an error
    df_adj_close_return.reset_index().drop(columns='Date'), 
    pd.read_csv('data/adj_close_return.txt'), 
    rtol=1e-5,
    atol=1e-8
)


*Note by ChatGPT:* <br>
if `pd.testing.assert_frame_equal()` doesn’t raise an exception, it means the two DataFrames are equal. If there are differences, it will raise an error detailing the mismatch.